In [1]:
import rclpy
#from rclpy.serialization import deserialize_message
#import rosbag2_py
from rosbag2_py import SequentialReader, StorageOptions, ConverterOptions
from cv_bridge import CvBridge
import cv2
import os
import sensor_msgs.msg
import rosbag2_py
from monoloco.predict import predict
import numpy as np


In [2]:
import sys
print(sys.executable)  

/Users/youssefelsaady/miniforge3/envs/ros_env/bin/python


In [3]:
rosbag_path = "/Users/youssefelsaady/Desktop/Practical Work/Optipex"

# Proceessing time
start_time = 1734616775.774053335
end_time = 1734616788.527853489

# extracted images
output_folder = "extracted_images"
os.makedirs(output_folder , exist_ok = True)

# Initializing ros_bag reader
reader = SequentialReader()
storage_options = StorageOptions(uri = rosbag_path , storage_id = 'sqlite3')
converter_options = ConverterOptions('', '')
reader.open(storage_options , converter_options)
filter = rosbag2_py.StorageFilter(topics=["/zed/zed_node/left/image_rect_color"])
reader.set_filter(filter)
# Geting topic information
topics = reader.get_all_topics_and_types()
bridge = CvBridge()


[INFO] [1742993691.523596004] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_0.db3' for READ_ONLY.


In [7]:
# Reading the messages from the ros_bag
#reader.set_filter([("/zed/zed_node/left/image_rect_color", "sensor_msgs/msg/Image")])
while reader.has_next():
    topic , data , timestamp = reader.read_next()
    msg = deserialize_message(data , sensor_msgs.msg.Image)
    # converting ros timestamp to seconds
    msg_time = msg.header.stamp.sec + msg.header.stamp.nanosec * 1e-9
    # Making sure the message is within the time range we need
    if start_time <= msg_time <= end_time:
        # converting Ros2 image to opencv format
        cv_image = bridge.imgmsg_to_cv2(msg , desired_encoding = "bgr8")
        # saveing the image as a PNG file
        image_filename = os.path.join(output_folder , f"{msg_time:.9f}.png")
        cv2.imwrite(image_filename , cv_image)
        print(f"Saved: {image_filename}")
print("Extraction complete!")

[INFO] [1742918415.321099763] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_1.db3' for READ_ONLY.


Saved: extracted_images/1734616775.774053335.png
Saved: extracted_images/1734616775.940945387.png
Saved: extracted_images/1734616776.074943542.png
Saved: extracted_images/1734616776.207899332.png
Saved: extracted_images/1734616776.374941349.png
Saved: extracted_images/1734616776.508472443.png
Saved: extracted_images/1734616776.642311335.png
Saved: extracted_images/1734616776.775894403.png
Saved: extracted_images/1734616776.942249537.png
Saved: extracted_images/1734616777.076903343.png
Saved: extracted_images/1734616777.242648363.png
Saved: extracted_images/1734616777.410064459.png
Saved: extracted_images/1734616777.543611526.png
Saved: extracted_images/1734616777.710915327.png
Saved: extracted_images/1734616777.844017506.png
Saved: extracted_images/1734616778.011349440.png
Saved: extracted_images/1734616778.144856453.png
Saved: extracted_images/1734616778.311420441.png
Saved: extracted_images/1734616778.445024490.png
Saved: extracted_images/1734616778.579141378.png
Saved: extracted_ima

[INFO] [1742918415.984827935] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_2.db3' for READ_ONLY.


Saved: extracted_images/1734616787.626941442.png
Saved: extracted_images/1734616787.760415554.png
Saved: extracted_images/1734616787.927043438.png
Saved: extracted_images/1734616788.060040474.png
Saved: extracted_images/1734616788.228708506.png
Saved: extracted_images/1734616788.394329548.png
Saved: extracted_images/1734616788.527853489.png


[INFO] [1742918416.353442234] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_3.db3' for READ_ONLY.
[INFO] [1742918416.549804301] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_4.db3' for READ_ONLY.
[INFO] [1742918416.732991507] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_5.db3' for READ_ONLY.
[INFO] [1742918416.916260421] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_6.db3' for READ_ONLY.
[INFO] [1742918417.110215808] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_7.db3' for READ_ONLY.
[INFO] [1742918417.305631620] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Desktop/Practical Work/Optipex/005_sabry_8.db3' for READ_ONLY.
[INFO] [1742918417.493407559] [rosbag2_storage]: Opened database '/Users/youssefelsaady/Deskto

RuntimeError: Error reading SQL query. SQLite error (11): database disk image is malformed

In [9]:
# Paths
image_dir = '/Users/youssefelsaady/Desktop/Practical Work/extracted_images/'  
output_dir = '/Users/youssefelsaady/Desktop/Practical Work/output_predictions/' 
os.makedirs(output_dir , exist_ok = True)
# Initializing the Monoloco model
#model = MonolocoModel()
# looping through the images & runing the model
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # doing the full image path
        image_path = os.path.join(image_dir , filename)
        # creating the output path for prediction
        output_path = os.path.join(output_dir , filename.split('.')[0])
        try:
            predict(
                image_path, 
                mode='mono',  
                output_dir = output_path, 
                long_edge = 1000 
            )
            print(f"Processed: {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

Error processing 1734616787.927043438.png: name 'predict' is not defined
Error processing 1734616785.289476395.png: name 'predict' is not defined
Error processing 1734616779.947765350.png: name 'predict' is not defined
Error processing 1734616778.879242420.png: name 'predict' is not defined
Error processing 1734616783.686670542.png: name 'predict' is not defined
Error processing 1734616786.457945347.png: name 'predict' is not defined
Error processing 1734616786.724496365.png: name 'predict' is not defined
Error processing 1734616786.558060408.png: name 'predict' is not defined
Error processing 1734616783.887075424.png: name 'predict' is not defined
Error processing 1734616778.011349440.png: name 'predict' is not defined
Error processing 1734616785.456094503.png: name 'predict' is not defined
Error processing 1734616780.114912510.png: name 'predict' is not defined
Error processing 1734616780.815389395.png: name 'predict' is not defined
Error processing 1734616781.883795500.png: name 'pr

In [9]:
import os
import monoloco.run
import logging

# Trying logging 
logging.basicConfig(level = logging.INFO)
# Paths
image_dir = '/Users/youssefelsaady/Desktop/Practical Work/extracted_images/'  
output_dir = '/Users/youssefelsaady/Desktop/Practical Work/output_predictions/'  
os.makedirs(output_dir , exist_ok = True)
# looping through the images & runing the model
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # doing the full image path
        image_path = os.path.join(image_dir , filename)     
        # creating the output path for prediction while saving the output with the same name but in output_dir
        output_path = os.path.join(output_dir , filename.split('.')[0])
        try:
            logging.info(f"Processing image: {filename}")
            predict(
                image_path,  
                mode='mono',  
                output_directory = output_path,  
                long_edge = 1000  
            )
            logging.info(f"Processed: {filename}")
        except Exception as e:
            logging.error(f"Error processing {filename}: {e}")

INFO:root:Processing image: 1734616787.927043438.png
ERROR:root:Error processing 1734616787.927043438.png: name 'predict' is not defined
INFO:root:Processing image: 1734616785.289476395.png
ERROR:root:Error processing 1734616785.289476395.png: name 'predict' is not defined
INFO:root:Processing image: 1734616779.947765350.png
ERROR:root:Error processing 1734616779.947765350.png: name 'predict' is not defined
INFO:root:Processing image: 1734616778.879242420.png
ERROR:root:Error processing 1734616778.879242420.png: name 'predict' is not defined
INFO:root:Processing image: 1734616783.686670542.png
ERROR:root:Error processing 1734616783.686670542.png: name 'predict' is not defined
INFO:root:Processing image: 1734616786.457945347.png
ERROR:root:Error processing 1734616786.457945347.png: name 'predict' is not defined
INFO:root:Processing image: 1734616786.724496365.png
ERROR:root:Error processing 1734616786.724496365.png: name 'predict' is not defined
INFO:root:Processing image: 1734616786.55

In [4]:
import os
import subprocess

# Paths
image_dir = '/Users/youssefelsaady/Desktop/Practical work/extracted_images/'
output_dir = '/Users/youssefelsaady/Desktop/Practical work/output_predictions/'
os.makedirs(output_dir , exist_ok = True)
# looping through the images & runing the model
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # doing the full image path
        image_path = os.path.join(image_dir , filename)
        # creating the output path for prediction
        output_path = os.path.join(output_dir , f"out_{filename}")
        # runing model in shell
        command = [
            "python3", "-m", "monoloco.run", "predict", 
            image_path, 
            "--output-directory", output_dir 
        ]
        try:
            result = subprocess.run(command, capture_output=True, text=True)
            # Check for any errors in the command output
            if result.returncode == 0:
                print(f"Processed: {filename}")
            else:
                print(f"Error processing {filename}: {result.stderr}")
        except Exception as e:
            print(f"Error executing command for {filename}: {e}")

Processed: 1734616787.927043438.png
Processed: 1734616785.289476395.png
Processed: 1734616779.947765350.png
Processed: 1734616778.879242420.png
Processed: 1734616783.686670542.png
Processed: 1734616786.457945347.png
Processed: 1734616786.724496365.png
Processed: 1734616786.558060408.png
Processed: 1734616783.887075424.png
Processed: 1734616778.011349440.png
Processed: 1734616785.456094503.png
Processed: 1734616780.114912510.png
Processed: 1734616780.815389395.png
Processed: 1734616781.883795500.png
Processed: 1734616778.745402336.png
Processed: 1734616783.319726467.png
Processed: 1734616784.721693516.png
Processed: 1734616787.626941442.png
Processed: 1734616784.054267406.png
Processed: 1734616776.775894403.png
Processed: 1734616782.952224493.png
Processed: 1734616782.217726469.png
Processed: 1734616786.157559395.png
Processed: 1734616779.179404497.png
Processed: 1734616778.311420441.png
Processed: 1734616776.074943542.png
Processed: 1734616784.555142403.png
Processed: 1734616781.182485

In [5]:
import os
import subprocess

# Paths
image_dir = '/Users/youssefelsaady/Desktop/Practical work/extracted_images/'
output_dir = '/Users/youssefelsaady/Desktop/Practical work/Output_of_trained_model/'
model_path = "/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output.pkl"
os.makedirs(output_dir , exist_ok = True)
# looping through the images & runing the model
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # doing the full image path
        image_path = os.path.join(image_dir , filename)
        # creating the output path for prediction
        output_path = os.path.join(output_dir , f"out_{filename}")
        # runing model in shell
        command = [
            "python3", "-m", "monoloco.run", "predict",
            image_path,
            "--mode", "mono",                  # using mono mode
            "--model", f"{model_path}",             # specifying the model path
            "--output-directory", output_dir   # specifying the output folder
        ]
        try:
            result = subprocess.run(command, capture_output=True, text=True)
            # Check for any errors in the command output
            if result.returncode == 0:
                print(f"Processed: {filename}")
            else:
                print(f"Error processing {filename}: {result.stderr}")
        except Exception as e:
            print(f"Error executing command for {filename}: {e}")

Error processing 1734616787.927043438.png: INFO:monoloco.predict:Force complete pose is active
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/youssefelsaady/Desktop/Practical work/Monoloco/monoloco/run.py", line 221, in <module>
    main()
  File "/Users/youssefelsaady/Desktop/Practical work/Monoloco/monoloco/run.py", line 150, in main
    predict(args)
  File "/Users/youssefelsaady/Desktop/Practical work/Monoloco/monoloco/predict.py", line 166, in predict
    net = Loco(
          ^^^^^
  File "/Users/youssefelsaady/Desktop/Practical work/Monoloco/monoloco/network/net.py", line 77, in __init__
    self.model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage))
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/youssefelsaady/miniforge3/envs/ros_env/lib/python3.11/site-packages/torch/serial

KeyboardInterrupt: 

In [7]:
from monoloco.network import Loco

# Initialize the model with the required parameters (assuming 'mono' mode is correct for your use case)
model = Loco(model='/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output.pkl', mode='mono')  # Adjust 'model' with the correct parameter

# Optionally, load checkpoint if needed
# model.load_state_dict(torch.load('<path_to_checkpoint>'))

# Save the model state dict after the training is complete
torch.save(model.state_dict(), "/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output.pth")
print("Model saved successfully!")

UnpicklingError: unpickling stack underflow

In [15]:
import torch
import monoloco.run   # Import the train function from monoloco.run

# Manually define the arguments (as done in the terminal command)
class Args:
    def __init__(self):
        self.joints = "/Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json"
        self.mode = 'mono'
        self.out = "/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output.pkl"  # Save model as .pkl
        self.e = 50  # Epochs
        self.bs = 8  # Batch size
        self.monocular = True  # Use monocular mode
        self.dropout = 0.2
        self.lr = 0.001  # Learning rate
        self.sched_step = 10  # Scheduler step
        self.sched_gamma = 0.5  # Scheduler gamma
        self.hidden_size = 512
        self.n_stage = 3
        self.print_loss = True  # Print loss
        self.device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")  # Set device
        self.log_file = "/Users/youssefelsaady/Desktop/Practical work/Train_output/training_log.txt"  # Save log

# Create the argument object
args = Args()

# Ensure the output model path is correct
print(f"Saving model to: {args.out}")

# Call the training function with the updated arguments
try:
    train(args)  # Call the train function from monoloco.run with the args object
    print(f"Training complete. Model saved to {args.out}")
except Exception as e:
    print(f"Training failed: {e}")

Saving model to: /Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output.pkl
Training failed: name 'train' is not defined


In [20]:
import subprocess

# Define the command as a list (same as your terminal command)
command = [
    "python3", "-m", "monoloco.run", "train",
    "--joints", "/Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json",
    "--mode", "mono",
    "--out", "/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output2.pkl",
    "-e", "50",  # Epochs
    "--bs", "8",  # Batch size
    "--monocular",
    "--dropout", "0.2",
    "--lr", "0.001",
    "--sched_step", "10",
    "--sched_gamma", "0.5",
    "--hidden_size", "512",
    "--n_stage", "3",
    "--print_loss",
]

# Add tee command to log output
log_file = "/Users/youssefelsaady/Desktop/Practical work/Train_output/training_log2.txt"
command += ["|", "tee", log_file]

# Execute the command
try:
    result = subprocess.run(command, check=True, text=True, shell=True)
    print("Training process completed.")
except subprocess.CalledProcessError as e:
    print(f"Error during training: {e}")

Training process completed.


In [40]:
import torch
from monoloco.monoloco.train import Trainer  # Import the Trainer class directly
from monoloco.run import cli  # Import the argument parsing function

# Manually define the arguments (similar to the command-line ones)
class Args:
    def __init__(self):
        # Arguments for training
        self.joints = '/Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json'
        self.mode = 'mono'
        self.out = '/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_outputtrial.pkl'
        self.epochs = 50
        self.bs = 8  # Batch size
        self.monocular = True
        self.dropout = 0.2
        self.lr = 0.001
        self.sched_step = 10
        self.sched_gamma = 0.5
        self.hidden_size = 1024
        self.n_stage = 3
        self.print_loss = True
        self.r_seed = 1  # Default random seed
        self.no_save = False

        # Add missing arguments
        self.auto_tune_mtl = True  # Set this to True or False as needed

args = Args()

# Check device and set it to 'mps' or 'cpu'
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Initialize the Trainer class and start training
trainer = Trainer(args, device)

# Run the training
trainer.train()
trainer.evaluate()

Using device: mps
/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_outputtrial.pkl
Device:  mps



VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training arguments:
mode: mono 
learning rate: 0.001 
batch_size: 8
epochs: 50 
dropout: 0.2 
scheduler step: 10 
scheduler gamma: 0.5 
input_size: 34 
output_size: 9 
hidden_size: 1024 
n_stages: 3 
 r_seed: 1 
lambdas: (1, 1, 1, 1, 1, 1, 1)

VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training arguments:
mode: mono 
learning rate: 0.001 
batch_size: 8
epochs: 50 
dropout: 0.2 
scheduler step: 10 
scheduler gamma: 0.5 
input_size: 34 
output_size: 9 
hidden_size: 1024 
n_stages: 3 
 r_seed: 1 
lambdas: (1, 1, 1, 1, 1, 1, 1)

VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training argumen

/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_outputtrial.pkl
>>> creating model
>>> model params: 8.458M
>>> loss params: 7
0 T::189.6  d:10507  x:4196  y:13093  h:4345  w:7929  l:4760  ori:49.0  V::185.2  d:10382  x:4113  y:12917  h:4295  w:7746  l:4703  ori:45.4  
10 T::186.8  d:10434  x:4117  y:13014  h:4266  w:7849  l:4679  ori:47.8  V::186.1  d:10395  x:4140  y:12938  h:4311  w:7770  l:4727  ori:45.2  
20 T::186.7  d:10431  x:4115  y:13011  h:4265  w:7848  l:4679  ori:47.9  V::186.0  d:10415  x:4144  y:12950  h:4327  w:7780  l:4736  ori:46.0  
30 T::186.7  d:10434  x:4119  y:13014  h:4267  w:7850  l:4680  ori:47.9  V::185.1  d:10387  x:4107  y:12917  h:4296  w:7740  l:4697  ori:46.1  
40 T::186.7  d:10433  x:4118  y:13014  h:4265  w:7849  l:4679  ori:47.8  V::186.4  d:10425  x:4159  y:12963  h:4338  w:7794  l:4751  ori:45.9  


Training:
Training complete in 5m 2s
Training:
Training complete in 5m 2s
Training:
Training complete in 5m 2s
Training:
Training complete in 5m 2s
Training:
Training complete in 5m 2s
Training:
Training complete in 5m 2s
Training:
Training complete in 5m 2s
Best training Accuracy: 186.686
Best training Accuracy: 186.686
Best training Accuracy: 186.686
Best training Accuracy: 186.686
Best training Accuracy: 186.686
Best training Accuracy: 186.686
Best training Accuracy: 186.686
Best validation Accuracy for d: 103.554
Best validation Accuracy for d: 103.554
Best validation Accuracy for d: 103.554
Best validation Accuracy for d: 103.554
Best validation Accuracy for d: 103.554
Best validation Accuracy for d: 103.554
Best validation Accuracy for d: 103.554
Saved weights of the model at epoch: 44
Saved weights of the model at epoch: 44
Saved weights of the model at epoch: 44
Saved weights of the model at epoch: 44
Saved weights of the model at epoch: 44
Saved weights of the model at epoch: 



------------------------------------------------------------------------------------------------------------------------


IndexError: list index out of range

In [26]:
import sys
sys.path.append('/Users/youssefelsaady/Desktop/Practical work/monoloco')  # Make sure to replace this with the correct path

In [42]:
import os
import subprocess

# Paths to your directories
image_dir = '/Users/youssefelsaady/Desktop/Practical work/extracted_images/'
output_dir = '/Users/youssefelsaady/Desktop/Practical work/Output_of_trained_model_final/'
model_path = "/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_model_final.pth"  # Make sure this is correct

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through images in the image directory
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # Full image path
        image_path = os.path.join(image_dir, filename)
        
        # Create output path for the prediction
        output_path = os.path.join(output_dir, f"out_{filename}")
        
        # Command to run the model for prediction
        command = [
            "python3", "-m", "monoloco.run", "predict",
            image_path,                         # Input image path
            "--mode", "mono",                    # Using mono mode
            "--model", f"{model_path}",          # Path to the trained model
            "--output-directory", output_dir    # Output directory for predictions
        ]
        
        # Execute the command
        try:
            result = subprocess.run(command, capture_output=True, text=True)
            
            # Check if the process was successful
            if result.returncode == 0:
                print(f"Processed: {filename}")
            else:
                print(f"Error processing {filename}: {result.stderr}")
        except Exception as e:
            print(f"Error executing command for {filename}: {e}")

Processed: 1734616787.927043438.png
Processed: 1734616785.289476395.png
Processed: 1734616779.947765350.png
Processed: 1734616778.879242420.png
Processed: 1734616783.686670542.png
Processed: 1734616786.457945347.png
Processed: 1734616786.724496365.png
Processed: 1734616786.558060408.png
Processed: 1734616783.887075424.png
Processed: 1734616778.011349440.png
Processed: 1734616785.456094503.png
Processed: 1734616780.114912510.png
Processed: 1734616780.815389395.png
Processed: 1734616781.883795500.png
Processed: 1734616778.745402336.png
Processed: 1734616783.319726467.png
Processed: 1734616784.721693516.png
Processed: 1734616787.626941442.png
Processed: 1734616784.054267406.png
Processed: 1734616776.775894403.png
Processed: 1734616782.952224493.png
Processed: 1734616782.217726469.png
Processed: 1734616786.157559395.png
Processed: 1734616779.179404497.png
Processed: 1734616778.311420441.png
Processed: 1734616776.074943542.png
Processed: 1734616784.555142403.png
Processed: 1734616781.182485

In [33]:
import torch
from monoloco.monoloco.train import Trainer  # Import the Trainer class directly
from monoloco.run import cli  # Import the argument parsing function

# Manually define the arguments (similar to the command-line ones)
class Args:
    def __init__(self):
        # Arguments for training
        self.joints = '/Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json'
        self.mode = 'mono'
        self.out = '/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output3.pkl'
        self.epochs = 50
        self.bs = 8  # Batch size
        self.monocular = True
        self.dropout = 0.2
        self.lr = 0.001
        self.sched_step = 10
        self.sched_gamma = 0.5
        self.hidden_size = 512
        self.n_stage = 3
        self.print_loss = True
        self.r_seed = 1  # Default random seed
        self.no_save = False
        self.auto_tune_mtl = True  # Set this to True or False as needed

        # Adding model name argument for naming the model
        self.model_name = 'my_trained_model'  # Default name

args = Args()

# Check device and set it to 'mps' or 'cpu'
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Initialize the Trainer class and pass the model name
trainer = Trainer(args, device)

# Run the training
trainer.train()

# Manually invoke the evaluation method if it doesn't work automatically
# Here we check for a potential missing `evaluate` method or any issues with its execution
try:
    trainer.evaluate()
except AttributeError:
    print("Evaluation method not implemented or not working, skipping.")

# Alternatively, after training, you can explicitly save the model if evaluate doesn't work.
model_path = f"/path/to/save/{args.model_name}.pth"
torch.save(trainer.model.state_dict(), model_path)
print(f"Model saved as {model_path}")

Using device: mps
/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output3.pkl
Device:  mps



VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training arguments:
mode: mono 
learning rate: 0.001 
batch_size: 8
epochs: 50 
dropout: 0.2 
scheduler step: 10 
scheduler gamma: 0.5 
input_size: 34 
output_size: 9 
hidden_size: 512 
n_stages: 3 
 r_seed: 1 
lambdas: (1, 1, 1, 1, 1, 1, 1)

VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training arguments:
mode: mono 
learning rate: 0.001 
batch_size: 8
epochs: 50 
dropout: 0.2 
scheduler step: 10 
scheduler gamma: 0.5 
input_size: 34 
output_size: 9 
hidden_size: 512 
n_stages: 3 
 r_seed: 1 
lambdas: (1, 1, 1, 1, 1, 1, 1)
Sizes of the dataset: {'train': 1600, 'val': 400}
Sizes of the dataset: {'train': 1600, 'val': 400}


/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output3.pkl
>>> creating model
>>> model params: 2.132M
>>> loss params: 7
0 T::200.9  d:10964  x:4624  y:13517  h:4759  w:8374  l:5210  ori:54.1  V::198.6  d:10908  x:4629  y:13388  h:4769  w:8258  l:5196  ori:51.7  
10 T::199.2  d:10935  x:4594  y:13486  h:4728  w:8344  l:5180  ori:53.8  V::198.6  d:10930  x:4642  y:13419  h:4796  w:8282  l:5223  ori:52.4  
20 T::199.1  d:10933  x:4591  y:13484  h:4725  w:8340  l:5177  ori:53.8  V::198.4  d:10915  x:4626  y:13397  h:4776  w:8261  l:5202  ori:51.9  
30 T::198.9  d:10930  x:4588  y:13481  h:4722  w:8337  l:5174  ori:53.7  V::198.8  d:10916  x:4638  y:13398  h:4776  w:8270  l:5203  ori:51.8  
40 T::199.1  d:10933  x:4590  y:13484  h:4725  w:8341  l:5178  ori:53.7  V::197.4  d:10893  x:4598  y:13372  h:4754  w:8235  l:5177  ori:51.9  


Training:
Training complete in 4m 18s
Training:
Training complete in 4m 18s
Best training Accuracy: 199.043
Best training Accuracy: 199.043
Best validation Accuracy for d: 108.665
Best validation Accuracy for d: 108.665
Saved weights of the model at epoch: 3
Saved weights of the model at epoch: 3




------------------------------------------------------------------------------------------------------------------------


IndexError: list index out of range

In [41]:
import torch
from monoloco.monoloco.train import Trainer  # Import the Trainer class directly
from monoloco.run import cli  # Import the argument parsing function

# Manually define the arguments (similar to the command-line ones)
class Args:
    def __init__(self):
        # Arguments for training
        self.joints = '/Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json'
        self.mode = 'mono'
        self.out = '/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output_final.pkl'
        self.epochs = 500
        self.bs = 8  # Batch size
        self.monocular = True
        self.dropout = 0.2
        self.lr = 0.001
        self.sched_step = 10
        self.sched_gamma = 0.5
        self.hidden_size = 1024
        self.n_stage = 3
        self.print_loss = True
        self.r_seed = 1  # Default random seed
        self.no_save = False
        self.auto_tune_mtl = True  # Set this to True or False as needed
        self.model_name = 'my_trained_model_final'  # Default name

args = Args()

# Check device and set it to 'mps' or 'cpu'
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Initialize the Trainer class and pass the model name
trainer = Trainer(args, device)

# Run the training
trainer.train()


# Alternatively, after training, you can explicitly save the model if evaluate doesn't work.
model_path = "/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_model_final.pth"
torch.save(trainer.model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Using device: mps
/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output_final.pkl
Device:  mps



VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training arguments:
mode: mono 
learning rate: 0.001 
batch_size: 8
epochs: 500 
dropout: 0.2 
scheduler step: 10 
scheduler gamma: 0.5 
input_size: 34 
output_size: 9 
hidden_size: 1024 
n_stages: 3 
 r_seed: 1 
lambdas: (1, 1, 1, 1, 1, 1, 1)

VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training arguments:
mode: mono 
learning rate: 0.001 
batch_size: 8
epochs: 500 
dropout: 0.2 
scheduler step: 10 
scheduler gamma: 0.5 
input_size: 34 
output_size: 9 
hidden_size: 1024 
n_stages: 3 
 r_seed: 1 
lambdas: (1, 1, 1, 1, 1, 1, 1)

VERSION: 0.7.4.6

INPUT_FILE: /Users/youssefelsaady/Desktop/Practical work/LSP Dataset/joints_with_images2.json
Input file version: lsp_custom
Torch version: 2.1.0

Training argum

/Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_output_final.pkl
>>> creating model
>>> model params: 8.458M
>>> loss params: 7
0 T::189.6  d:10507  x:4196  y:13093  h:4345  w:7929  l:4760  ori:49.0  V::185.2  d:10382  x:4113  y:12917  h:4295  w:7746  l:4703  ori:45.4  
10 T::186.8  d:10434  x:4117  y:13014  h:4266  w:7849  l:4679  ori:47.8  V::186.1  d:10395  x:4140  y:12938  h:4311  w:7770  l:4727  ori:45.2  
20 T::186.7  d:10431  x:4115  y:13011  h:4265  w:7848  l:4679  ori:47.9  V::186.0  d:10415  x:4144  y:12950  h:4327  w:7780  l:4736  ori:46.0  
30 T::186.7  d:10434  x:4119  y:13014  h:4267  w:7850  l:4680  ori:47.9  V::185.1  d:10387  x:4107  y:12917  h:4296  w:7740  l:4697  ori:46.1  
40 T::186.7  d:10433  x:4118  y:13014  h:4265  w:7849  l:4679  ori:47.8  V::186.4  d:10425  x:4159  y:12963  h:4338  w:7794  l:4751  ori:45.9  
50 T::186.8  d:10438  x:4121  y:13019  h:4269  w:7855  l:4684  ori:47.8  V::185.9  d:10404  x:4135  y:12941  h:4317  w:7771  l:472

Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Training:
Training complete in 53m 14s
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best training Accuracy: 186.742
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Best validation Accuracy for d: 103.284
Saved weights of the model at epoch: 118
Saved weights of the model at epoch: 118
Saved weights of the model at 



------------------------------------------------------------------------------------------------------------------------
Model saved to /Users/youssefelsaady/Desktop/Practical work/Train_output/monoloco_lsp_model_final.pth
